## susieR


需要的数据：
- Zscore of GWAS 
- var y 
- n

In [19]:
data_dir = "/home/xutingfeng/GIFT/"  # 数据存放路径
gwas_pheno = "/home/xutingfeng/ukb/ukbData/phenotypes/kidney"
gwas_file = data_dir + "data/GWAS/T1Mapping_Cortex_20240129.csv_firstorder_Median_all_2023_GRCh38_unionKidneys.tsv.gz"
# pfile = "/home/xutingfeng/ukb/externel/1kg/g1000_eur_GRCh38"
pfile = "/home/xutingfeng/GIFT/data/pgen"
pheno_path = "/home/xutingfeng/ukb/ukbData/phenotypes/kidney/union_v3_INT.tsv"

In [2]:
import pandas as pd 
import h5py
from finemap_tools.plink import plink2_cal_LD
from finemap_tools.reader.gwas import load_GWASFormated_file

In [20]:
pheno = pd.read_csv(pheno_path, sep = "\s+")[["T1Mapping_Cortex_20240129.csv_firstorder_Median"]]
var_y = pheno.std().iloc[0]
var_y

0.999606259706267

In [4]:
topLoci = (1, 15583355, 15583355) # (chr, start, end)
locus_range = 250 # 100kb 
locus_range = locus_range * 1000 
locus_range_tuple = (topLoci[0], topLoci[1] - locus_range, topLoci[2] + locus_range)

locus_region = f"{locus_range_tuple[0]}:{locus_range_tuple[1]}-{locus_range_tuple[2]}"
print(f"locus_region: {locus_region}")

locus_region: 1:15333355-15833355


In [5]:
from pathlib import Path
from finemap_tools.utils import add_ID
from finemap_tools.snpfilter.snp import find_Biallelic_snp, is_ambiguous_alleles


gwas = load_GWASFormated_file(file_path = gwas_file, region = locus_region )
gwas['ID'] = add_ID(gwas, col_list=["chrom", "pos", "ref", "alt"])

# 1. filter out 
from finemap_tools.snpfilter.snp import filter_pipline
clean = filter_pipline(gwas['ID'], id_sep=':')
gwas = gwas[gwas['ID'].isin(clean)]
gwas 

tabix have a header, so will take the first line as header and remove it.
drop 463 ambiguous alleles
drop 10 biallelic snps


,chrom,pos,ref,alt,pval,af,beta,sebeta,ID
1,1,15333718,C,T,0.4508499231676181,0.017148,0.0556581,0.0738172,1:15333718:C:T
2,1,15334029,G,A,0.03889466184217771,0.00101557,0.694909,0.336469,1:15334029:G:A
3,1,15334133,G,T,0.9375976280715118,0.315017,-0.00163464,0.0208793,1:15334133:G:T
4,1,15334153,C,T,0.41442695989345996,0.0173697,0.059858,0.0733439,1:15334153:C:T
5,1,15334177,A,G,0.9226311218938984,0.316113,-0.00202684,0.0208694,1:15334177:A:G
...,...,...,...,...,...,...,...,...,...
3996,1,15832572,C,T,0.10525457936702215,0.0843496,0.0563468,0.0347842,1:15832572:C:T
3997,1,15832764,A,AT,0.18805245177296775,0.33544,-0.0273396,0.020769,1:15832764:A:AT
3998,1,15833104,T,C,0.9634930804152086,0.000564717,-0.0185399,0.405061,1:15833104:T:C
3999,1,15833289,G,A,0.1555045138969292,0.000676743,0.55788,0.392775,1:15833289:G:A


In [6]:
from finemap_tools.reader.plink import read_pvar
pvar = read_pvar(pfile + ".pvar")
pvar 

/home/xutingfeng/github_code/mine/finemap_tools/finemap_tools/reader/plink/pfile.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(pvar, sep="\t", skiprows=comment_lines)


,#CHROM,POS,ID,REF,ALT
0,1,10539,1:10539:C:A,C,A
1,1,11008,1:11008:C:G,C,G
2,1,11012,1:11012:C:G,C,G
3,1,13110,1:13110:G:A,G,A
4,1,13116,1:13116:T:G,T,G
...,...,...,...,...,...
22665059,X,155699751,X:155699751:C:T,C,T
22665060,X,155699978,X:155699978:T:C,T,C
22665061,X,155700569,X:155700569:A:G,A,G
22665062,X,195552978,X:195552978:C:T,C,T


In [7]:
from finemap_tools.utils import sort_ID
pvar['sort_ID'] = sort_ID(pvar, id_col = "ID")
gwas['sort_ID'] = sort_ID(gwas, id_col = "ID")

# 2. merge

intersect_sort_ID = set(pvar['sort_ID']).intersection(set(gwas['sort_ID']))
print(f"intersect_sort_ID: {len(intersect_sort_ID)}")

intersect_sort_ID: 2278


In [8]:
gwas = gwas[gwas['sort_ID'].isin(intersect_sort_ID)]
pvar = pvar[pvar['sort_ID'].isin(intersect_sort_ID)]

snplist = set(pvar['ID'].tolist())

In [9]:
del gwas["ID"]
gwas = gwas.merge(pvar[['ID', 'sort_ID']], on = "sort_ID")

In [10]:
# gwas = gwas.drop_duplicates(subset = "ID")
gwas 

,chrom,pos,ref,alt,pval,af,beta,sebeta,sort_ID,ID
0,1,15333718,C,T,0.4508499231676181,0.017148,0.0556581,0.0738172,1:15333718:C:T,1:15333718:C:T
1,1,15334133,G,T,0.9375976280715118,0.315017,-0.00163464,0.0208793,1:15334133:G:T,1:15334133:G:T
2,1,15334153,C,T,0.41442695989345996,0.0173697,0.059858,0.0733439,1:15334153:C:T,1:15334153:C:T
3,1,15334177,A,G,0.9226311218938984,0.316113,-0.00202684,0.0208694,1:15334177:A:G,1:15334177:A:G
4,1,15334558,C,T,0.3170056480082673,0.00203851,-0.213707,0.213572,1:15334558:C:T,1:15334558:C:T
...,...,...,...,...,...,...,...,...,...,...
2273,1,15831703,G,A,0.11313370239294869,0.00417607,0.249632,0.15757,1:15831703:A:G,1:15831703:G:A
2274,1,15832113,G,A,0.4876901678258844,0.0223982,-0.0447152,0.0644323,1:15832113:A:G,1:15832113:G:A
2275,1,15832572,C,T,0.10525457936702215,0.0843496,0.0563468,0.0347842,1:15832572:C:T,1:15832572:C:T
2276,1,15833289,G,A,0.1555045138969292,0.000676743,0.55788,0.392775,1:15833289:A:G,1:15833289:G:A


In [11]:
import tempfile
with tempfile.TemporaryDirectory() as tmpdirname:
    print('created temporary directory', tmpdirname)
    LD = plink2_cal_LD(pgen=pfile, snplist=snplist, outSuffix=tmpdirname + '/test')


created temporary directory /tmp/tmpf4kl4b89


In [12]:
LD

,1:15333718:C:T,1:15334133:G:T,1:15334153:C:T,1:15334177:A:G,1:15334558:C:T,1:15334603:G:A,1:15334808:C:T,1:15335582:C:T,1:15336110:C:T,1:15336203:G:A,...,1:15829911:C:T,1:15830433:G:A,1:15830447:G:A,1:15830723:C:T,1:15831227:C:T,1:15831703:G:A,1:15832113:G:A,1:15832572:C:T,1:15833289:G:A,1:15833295:A:C
1:15333718:C:T,1.000000,0.031944,1.000000,0.036527,0.000315,0.000315,0.032557,0.001119,0.010894,0.000236,...,0.000342,0.000078,0.000554,0.000342,0.000368,0.000474,0.002677,0.000368,0.000157,0.000342
1:15334133:G:T,0.031944,1.000000,0.031944,0.979320,0.008317,0.000047,0.983326,0.026665,0.822176,0.005163,...,0.013852,0.000388,0.000008,0.013852,0.003595,0.005655,0.005686,0.003595,0.000333,0.013852
1:15334153:C:T,1.000000,0.031944,1.000000,0.036527,0.000315,0.000315,0.032557,0.001119,0.010894,0.000236,...,0.000342,0.000078,0.000554,0.000342,0.000368,0.000474,0.002677,0.000368,0.000157,0.000342
1:15334177:A:G,0.036527,0.979320,0.036527,1.000000,0.008171,0.000042,0.979520,0.025896,0.813174,0.005012,...,0.010245,0.000374,0.000006,0.010245,0.003640,0.005477,0.004121,0.003640,0.000332,0.010245
1:15334558:C:T,0.000315,0.008317,0.000315,0.008171,1.000000,0.000064,0.008076,0.000213,0.007876,0.000048,...,0.014099,0.000016,0.000113,0.014099,0.001108,0.000097,0.000547,0.001108,0.000032,0.014099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:15831703:G:A,0.000474,0.005655,0.000474,0.005477,0.000097,0.038548,0.005810,0.000320,0.006340,0.000072,...,0.000063,0.000024,0.000170,0.000063,0.001668,1.000000,0.001769,0.001668,0.330672,0.000063
1:15832113:G:A,0.002677,0.005686,0.002677,0.004121,0.000547,0.000547,0.005203,0.002946,0.004464,0.000409,...,0.010984,0.000136,0.000962,0.010984,0.000823,0.001769,1.000000,0.000823,0.011140,0.010984
1:15832572:C:T,0.000368,0.003595,0.000368,0.003640,0.001108,0.001108,0.003076,0.002602,0.000455,0.000829,...,0.003873,0.000275,0.001950,0.003873,1.000000,0.001668,0.000823,1.000000,0.000552,0.003873
1:15833289:G:A,0.000157,0.000333,0.000157,0.000332,0.000032,0.000032,0.000305,0.000106,0.000261,0.000024,...,0.002109,0.000008,0.000056,0.002109,0.000552,0.330672,0.011140,0.000552,1.000000,0.002109


In [13]:
gwas['chrom'] = gwas['chrom'].astype(int)
gwas['pos'] = gwas['pos'].astype(int)
gwas['ID'] = gwas['ID'].astype(str)
gwas['ref'] = gwas['ref'].astype(str)
gwas['alt'] = gwas['alt'].astype(str)
gwas['pval'] = gwas['pval'].astype(float)
gwas['beta'] = gwas['beta'].astype(float)
gwas['sebeta'] = gwas['sebeta'].astype(float)
gwas['af'] = gwas['af'].astype(float)

In [14]:
assert all(gwas["ID"] == LD.columns) == True, "ID not equal"

In [15]:
len(LD.columns.tolist()), len(set(LD.columns.tolist()))

(2278, 2278)

In [16]:
LD.reset_index(inplace = True, drop=True)
LD.columns = LD.index

In [17]:
print(f"LD has na {LD.isna().sum().sum()}")
LD = LD.fillna(0)

LD has na 0


In [18]:
gwas.to_feather("/home/xutingfeng/GIFT/gwas.feather")
LD.to_feather("/home/xutingfeng/GIFT/LD.feather")

In [23]:
LD

,0,1,2,3,4,5,6,7,8,9,...,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277
0,1.000000,0.031944,1.000000,0.036527,0.000315,0.000315,0.032557,0.001119,0.010894,0.000236,...,0.000342,0.000078,0.000554,0.000342,0.000368,0.000474,0.002677,0.000368,0.000157,0.000342
1,0.031944,1.000000,0.031944,0.979320,0.008317,0.000047,0.983326,0.026665,0.822176,0.005163,...,0.013852,0.000388,0.000008,0.013852,0.003595,0.005655,0.005686,0.003595,0.000333,0.013852
2,1.000000,0.031944,1.000000,0.036527,0.000315,0.000315,0.032557,0.001119,0.010894,0.000236,...,0.000342,0.000078,0.000554,0.000342,0.000368,0.000474,0.002677,0.000368,0.000157,0.000342
3,0.036527,0.979320,0.036527,1.000000,0.008171,0.000042,0.979520,0.025896,0.813174,0.005012,...,0.010245,0.000374,0.000006,0.010245,0.003640,0.005477,0.004121,0.003640,0.000332,0.010245
4,0.000315,0.008317,0.000315,0.008171,1.000000,0.000064,0.008076,0.000213,0.007876,0.000048,...,0.014099,0.000016,0.000113,0.014099,0.001108,0.000097,0.000547,0.001108,0.000032,0.014099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,0.000474,0.005655,0.000474,0.005477,0.000097,0.038548,0.005810,0.000320,0.006340,0.000072,...,0.000063,0.000024,0.000170,0.000063,0.001668,1.000000,0.001769,0.001668,0.330672,0.000063
2274,0.002677,0.005686,0.002677,0.004121,0.000547,0.000547,0.005203,0.002946,0.004464,0.000409,...,0.010984,0.000136,0.000962,0.010984,0.000823,0.001769,1.000000,0.000823,0.011140,0.010984
2275,0.000368,0.003595,0.000368,0.003640,0.001108,0.001108,0.003076,0.002602,0.000455,0.000829,...,0.003873,0.000275,0.001950,0.003873,1.000000,0.001668,0.000823,1.000000,0.000552,0.003873
2276,0.000157,0.000333,0.000157,0.000332,0.000032,0.000032,0.000305,0.000106,0.000261,0.000024,...,0.002109,0.000008,0.000056,0.002109,0.000552,0.330672,0.011140,0.000552,1.000000,0.002109


In [25]:
# save to h5 
import h5py

with h5py.File("susieR.h5", "w") as f:
    f.create_group("data")
    f['data'].create_dataset('gwas', data=gwas)
    f['data'].create_dataset('LD', data=LD)
    f['data'].create_dataset('var_y', data=var_y)
    f['data'].create_dataset('topLoci', data=topLoci)
    f['data'].create_dataset("n", data = len(pheno))


TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [55]:
len(pheno)

5693